In [1]:
import yaml

with open('schema.yaml', 'r') as f:
    col = yaml.safe_load(f)

In [2]:
start = 1
awk_input = []
for c, width in col:
    if not c.startswith('FILLER'):
        awk_input.append(f'substr($0,{start},{width})')
    start += width

awk_input = ', '.join(awk_input)

In [3]:
import glob, re, os

inputs = glob.glob('data/raw/*.txt')
years = [re.compile(r'Nat([0-9]{4})PublicUS').search(i).group(1) for i in inputs]
col_final = '|'.join([c for c, _ in col if not c.startswith('FILLER')])

for y, i in zip(years, inputs):
    if os.path.exists(f'data/processed/births{y}.txt'):
        continue
    !awk -v OFS='|' '{{ print $awk_input }}' $i > data/processed/births{y}.txt
    !sed -i "1i $col_final" data/processed/births{y}.txt

In [4]:
!gsutil -m cp -n -r data/processed/*.txt gs://mother-goose-data/nvss/

Skipping existing item: gs://mother-goose-data/nvss/births2019.txt
Skipping existing item: gs://mother-goose-data/nvss/births2017.txt
Skipping existing item: gs://mother-goose-data/nvss/births2018.txt
Skipping existing item: gs://mother-goose-data/nvss/births2015.txt
Skipping existing item: gs://mother-goose-data/nvss/births2016.txt
Copying file://data/processed/births2014.txt [Content-Type=text/plain]...


In [5]:
from google.cloud import bigquery

client = bigquery.Client()

schema = [bigquery.SchemaField(c, "INTEGER" if c == 'DOB_YY' else "STRING") \
          for c, _ in col if not c.startswith('FILLER')]
table = bigquery.Table('mother-goose-health.nvss.births', schema=schema)
table.range_partitioning = bigquery.RangePartitioning(
    field="DOB_YY",
    range_=bigquery.PartitionRange(start=1900, end=2100, interval=1),
)

client.delete_table('mother-goose-health.nvss.births', not_found_ok=True)
table = client.create_table(table)

job_config = bigquery.LoadJobConfig(
    schema=schema,
    skip_leading_rows=1,
    source_format=bigquery.SourceFormat.CSV,
    field_delimiter='|'
)

load_job = client.load_table_from_uri(
    'gs://mother-goose-data/nvss/*.txt', 
    table, 
    job_config=job_config
)
load_job.result()

In [6]:
q = """
SELECT DOB_YY,
    REGEXP_REPLACE(SPLIT(pair, ':')[OFFSET(0)], r'^"|"$', '') metric, 
    REGEXP_REPLACE(SPLIT(pair, ':')[OFFSET(1)], r'^"|"$', '') value,
    COUNT(*) AS n
FROM nvss.births t, 
UNNEST(SPLIT(REGEXP_REPLACE(to_json_string(t), r'{|}', ''))) pair
GROUP BY 1,2,3
"""
df = client.query(q).to_dataframe()

In [7]:
exclude_col = ['BMI', 'CIG_0', 'CIG_1', 'CIG_2', 'CIG_3', 'DBWT', 'DOB_YY', 'DLMP_YY', 'FAGECOMB', 
               'M_HT_IN', 'PREVIS', 'PRIORDEAD', 'PRIORLIVE', 'PRIORTERM', 'RF_CESARN']

n_year = df.DOB_YY.nunique()

(
    df
    .query('value.fillna("").str.strip() != ""')
    .query('~metric.isin(@exclude_col)')
    .groupby(['metric', 'value'], as_index=False)
    .size()
    .query('size < @n_year')
    .groupby(['metric']).size()
)

metric
FHISPX      8
LMPUSED     1
MHISPX      8
MRACE15     1
MRACE31     1
MRACE6      1
MRACEIMP    1
dtype: int64